In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("heart.csv")

In [ ]:
print(data.head())


   age  sex  cp  trestbps  chol  fbs  ...  exang  oldpeak  slope  ca  thal  target
0   63    1   3       145   233    1  ...      0      2.3      0   0     1       1
1   37    1   2       130   250    0  ...      0      3.5      0   0     2       1
2   41    0   1       130   204    0  ...      0      1.4      2   0     2       1
3   56    1   1       120   236    0  ...      0      0.8      2   0     2       1
4   57    0   0       120   354    0  ...      1      0.6      2   0     2       1

[5 rows x 14 columns]


In [ ]:
data = data.apply(pd.to_numeric)
data.dtypes


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [ ]:
X =np.array(data.drop(["target"],axis=1))
y = np.array(data["target"])


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=45)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
from keras.utils.np_utils import to_categorical

Y_train = to_categorical(y_train, num_classes=None)
Y_test = to_categorical(y_test, num_classes=None)

In [ ]:
X_train[0]

array([ 1.24659003, -1.42472831,  1.01901457,  0.8191292 ,  0.60374615,
       -0.44853476, -0.95972915,  0.18008212, -0.72547625, -0.91101584,
       -0.57790793,  0.23569702, -0.45592706])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import regularizers

# define a function to build the keras model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=13, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(8, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(2, activation='softmax'))
    
    # compile model
    adam = Adam(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

model = create_model()

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                224       
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 136       
_________________________________________________________________
dropout_10 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 18        
Total params: 378
Trainable params: 378
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history=model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=35, batch_size=10)

Epoch 1/35
21/21 [==============================] - 0s 10ms/step - loss: 0.6754 - accuracy: 0.7094 - val_loss: 0.6646 - val_accuracy: 0.7400
Epoch 2/35
21/21 [==============================] - 0s 4ms/step - loss: 0.6367 - accuracy: 0.7882 - val_loss: 0.6265 - val_accuracy: 0.7700
Epoch 3/35
21/21 [==============================] - 0s 5ms/step - loss: 0.5950 - accuracy: 0.8128 - val_loss: 0.5850 - val_accuracy: 0.8100
Epoch 4/35
21/21 [==============================] - 0s 5ms/step - loss: 0.5373 - accuracy: 0.8325 - val_loss: 0.5424 - val_accuracy: 0.8400
Epoch 5/35
21/21 [==============================] - 0s 4ms/step - loss: 0.4967 - accuracy: 0.8325 - val_loss: 0.5062 - val_accuracy: 0.8500
Epoch 6/35
21/21 [==============================] - 0s 4ms/step - loss: 0.4565 - accuracy: 0.8473 - val_loss: 0.4774 - val_accuracy: 0.8500
Epoch 7/35
21/21 [==============================] - 0s 4ms/step - loss: 0.4449 - accuracy: 0.8424 - val_loss: 0.4540 - val_accuracy: 0.8500
Epoch 8/35
21/21 [=

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
categorical_pred = np.argmax(model.predict(X_test), axis=1)

print('Results for Categorical Model')
print(accuracy_score(y_test, categorical_pred))
print(classification_report(y_test, categorical_pred))

Results for Categorical Model
0.86
              precision    recall  f1-score   support

           0       0.85      0.80      0.83        41
           1       0.87      0.90      0.88        59

    accuracy                           0.86       100
   macro avg       0.86      0.85      0.85       100
weighted avg       0.86      0.86      0.86       100



In [ ]:
model.save("86_acc.h5")